In [1]:
pip install pytorch-gradcam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 17.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/f5/d0/2e455d894ec0d6527e662ad55e70c04f421ad83a6fd0a54c3dd73c411282/opencv_python-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 39.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for pytorch-gradcam: filename=pytorch_gradcam-0.2.1-py3-none-any.whl size=5246 sha256=1a585a34d7438522cbff1ef0e191075f38a03df2d3fefe21beccbf5f2280a93f
  Stored in directory: /home/magali/.cache/pip/wheels/dd/3c/0e/8b7512f5d7f462c9c37342107b7f0276609345f246ae4e75fc
Successfully built pytorch-gradcam
Note: you may need to restart the kernel to use updated packages.


In [ ]:
model = MyModel()
target_layer = model.my_submodule
gradcam = GradCAM(model, target_layer)